In [1]:
#This is an attempt for the final version of "v1", eventually to work with the repo data

In [2]:
#This version contains both NB (Naive Bayes) and SGD (Support Vector Machine)

In [1]:
#In this version, the 20newsgroup data are being used for now. 

In [61]:
from sklearn.datasets import fetch_20newsgroups

In [62]:
from sklearn.feature_extraction.text import CountVectorizer

In [63]:
from sklearn.linear_model import SGDClassifier

In [64]:
from sklearn.naive_bayes import MultinomialNB

In [65]:
from sklearn.pipeline import Pipeline

In [66]:
from sklearn.feature_extraction.text import TfidfTransformer

In [67]:
from sklearn.model_selection import GridSearchCV

In [68]:
import numpy as np

In [69]:
#Fetch data and store them in appropriate arrays

In [ ]:
import requests

API_URL = <DB_URL>

In [ ]:
#get the categories
response = requests.get(API_URL + "categories")
if response.status_code == 200:
    categories = [category["name"] for category in response.json()]
    categories += ["racial slur"] #add additional not in the API manually

In [ ]:
#Fetch data and store them in appropriate arrays
response = requests.get(API_URL + "mark")
if response.status_code == 200:
    raw_data = response.json()
    
response.status_code

In [ ]:
from collections import namedtuple
data_set = namedtuple('data_set', ["data", "urls", "target_names", "target"])([],[],categories,[])

for row in raw_data:
    if row['category'] in categories:
        data_set.data.append(row["flagged_string"])
        data_set.urls.append(row["url"])
        data_set.target.append(categories.index(row['category'])

In [ ]:
marker_train = data_set

In [ ]:
#For now using the same (training) data as the test data. Eventually, a separate test data set should be used.

In [ ]:
marker_test = data_set

In [ ]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

In [74]:
#marker_train.data is an array of strings (containing as many "texts" as there are coming from Marker.)

In [75]:
#marker_train.target is an array of integers (containing the class indices for the above texts' class labels)

In [76]:
#marker_train_target_names is an array of class label names indexed by the array indices.

In [77]:
#flagger_new_text = fetch_20newsgroups(subset='test',
#                                      categories=categories, shuffle=True, random_state=42)

In [78]:
marker_train.target.shape

(2257,)

In [79]:
marker_test.target.shape

(1502,)

In [80]:
len(marker_test.data)

1502

In [81]:
for t in marker_train.target[:10]:
...     print(marker_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


In [82]:
for t in marker_test.target[:10]:
...     print(marker_train.target_names[t])

sci.med
sci.med
sci.med
alt.atheism
soc.religion.christian
alt.atheism
comp.graphics
soc.religion.christian
sci.med
sci.med


In [83]:
#Classes for model representations packaging the components in the modeling pipeline.
#For SGDClassifier and NaiveBayes.
#We may want to extend it to utilize "Pipeline()" as in "ModelRepSGD0", which is not in use at the moment.

In [84]:
class ModelRepSGD:
    def __init__(self, count_vect, X_train_counts, tf_transformer, text_clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                  alpha=1e-3, random_state=42,
                                  max_iter=5, tol=None)),
        ])

In [85]:
class ModelRepNB:
    def __init__(self, count_vect, X_train_counts, tf_transformer, clf):
        self.count_vect = CountVectorizer()
        self.X_train_counts = self.count_vect.fit_transform(marker_train.data)
        self.tf_transformer = TfidfTransformer(use_idf=False).fit(self.X_train_counts)
        self.clf = MultinomialNB()

In [86]:
#Functions for model training - for SGDClassifier and NaiveBayes (SGD one is untested)

In [87]:
def model_train_SGD(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)),
    ])
    text_clf.fit(marker_train.data, marker_train.target)
    trained_model = ModelRepSGD(count_vect, X_train_counts, tf_transformer, text_clf)
    trained_model.text_clf = text_clf
    return trained_model

In [88]:
def model_train_NB(marker_train): 
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(marker_train.data)
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    clf = MultinomialNB().fit(X_train_tf, marker_train.target)
    trained_model = ModelRepNB(count_vect, X_train_counts, tf_transformer, clf)
    trained_model.clf = clf
    return trained_model

In [89]:
#Function for model testing (common for SGDClassifier and NB)

In [90]:
def model_test(flagger_new_test, trained_model):
    count_vect = trained_model.count_vect
    tf_transformer = trained_model.tf_transformer
    clf = trained_model.clf
    X_new_counts = count_vect.transform(flagger_new_test)
    X_new_tfidf = tf_transformer.transform(X_new_counts)
    predicted = clf.predict(X_new_tfidf)
    return predicted

In [91]:
#Now testing model_train and model_test

In [92]:
trained_model_NB = model_train_NB(marker_train)

In [93]:
trained_model_SGD = model_train_SGD(marker_train)

In [94]:
#Inspecting the components of the trained model

In [95]:
trained_model_NB.clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [96]:
trained_model_NB.count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [97]:
trained_model_NB.tf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=False)

In [98]:
trained_model_NB.clf

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [99]:
trained_model_SGD.count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [100]:
trained_model_SGD.tf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=False)

In [101]:
trained_model_SGD.text_clf

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=0.001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [102]:
trained_model_SGD.count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [103]:
trained_model_SGD.tf_transformer

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=False)

In [104]:
#Calling model tester/predictor on a single new text from Flagger

In [105]:
flagger_new_text = ['OpenGL on the GPU is fast']

In [106]:
predicted_NB = model_test(flagger_new_text, trained_model_NB)

In [107]:
predicted_SGD = trained_model_SGD.text_clf.predict(flagger_new_text)

In [108]:
predicted_NB

array([1])

In [109]:
predicted_SGD

array([1])

In [110]:
#Now evaluating NB and SGD models on a test data set

In [111]:
predicted_test_NB = model_test(marker_test.data, trained_model_NB)

In [112]:
predicted_test_SGD = trained_model_SGD.text_clf.predict(marker_test.data)

In [113]:
predicted_test_SGD.shape

(1502,)

In [114]:
predicted_test_SGD

array([2, 2, 2, ..., 2, 2, 1])

In [115]:
marker_test.target

array([2, 2, 2, ..., 2, 2, 1])

In [116]:
np.mean(predicted_test_NB == marker_test.target)

0.7669773635153129

In [117]:
np.mean(predicted_test_SGD == marker_test.target)

0.9101198402130493